In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
import list_initial_conditions as ic
import list_ibvp_solution as ibvp
import list_swwe_function as swe
from time import time
from flowtype import flowtype_function

start=time()
print("Start convergence test") 
# set up nature of the simulation, with or without manufactured solution:
# 1. manufactured_solution = ibvp.linearSWWE_mms
# 2. manufactured_solution = False
mms = ibvp.nonlinearSWWE_mms
number_of_iteration = 9

SAT_function = swe.linearised_SWWE_SAT_terms
RHS = swe.nonlinear_swwe_RHS

# set 'generated_wave = False' if manufactured_solution is not False
generated_wave = False
# generated_wave = ibvp.zero_wave
# generated_wave = ibvp.gaussian_wave_1
# generated_wave = ibvp.step_function_wave


# set up parameters
cfl = 0.25
H = 1.0
g = 9.81
c = np.sqrt(g*H)

# coefficient may vary 
U_coeff = 0.5
alpha_coeff = 0.0

U = U_coeff*c
alpha = alpha_coeff*(np.abs(U)+c)

flowtype = flowtype_function(U,c)

error_h = np.zeros(number_of_iteration)
error_u = np.zeros(number_of_iteration)

now = time()
for n__ in range(number_of_iteration):
        # set up the domain 
        # x0, xN = 0, 4
        x0, xN = 0,(np.abs(U)+np.sqrt(g*H))*5
        N = 2**(4+n__)
        print(f"N0: {n__}. Number of grid points: {N}")
        dx = (xN-x0)/(N-1)  
        x = np.linspace(x0,xN,N)

        # set up time parameters
        dt = cfl*dx/(np.abs(U) + np.sqrt(g*H))      # (cfl * dx)/max speed
        sim_time = np.pi*0.2

        # set up supportive matrix 
        Q, A, P_inv, I_N = swe.linearised_SWWE_matrix_supportive(N, dx)

        # list constants
        constants = x,H,0,g,c,alpha,flowtype,Q,A,P_inv, I_N, \
                RHS, SAT_function, \
                mms, generated_wave 

        # x,H,U,g,c,alpha,flowtype,Q,A,P_inv,RHS_function, SAT,manufactured_solution, analytical_solution = constants

        # solving the IBVP 

        # set up initial conditions
        q, _ = ibvp.nonlinearSWWE_mms(x,0,H,U,g)
        q_numerical = copy(q)
        q_analytical = copy(q)

        q_numerical = swe.numerical_solution_nonlinear(q_numerical,sim_time,dt,constants)
        q_analytical = swe.analytical_solution_mms(x,sim_time,dt,constants)

        now = time()
        
        # error calculation
        q_numerical = np.array(q_numerical[-1])
        q_analytical = np.array(q_analytical[-1])

        print(f'some h: {q_numerical[0][:5]}')

        error_h[n__] = np.linalg.norm(q_numerical[0] - q_analytical[0])*np.sqrt(dx)
        error_u[n__] = np.linalg.norm(q_numerical[1] - q_analytical[1])*np.sqrt(dx)

        

error_rate_h = np.log2(error_h[:-1]/error_h[1:])
error_rate_u = np.log2(error_h[:-1]/error_h[1:])


end = time()

print(f"Number of iteration: {number_of_iteration}, {2**(6)} to  {2**(6+n__)}, dx: {dx}")
print(f"simulation time: {np.round(sim_time,4)} s, dt: {np.round(dt,4)} s")
print(f"U*sqrt(gH): {U_coeff}, alpha*(np.abs(U)+c): {alpha_coeff}")
print(f"U: {U}, alpha: {alpha}")
print(f"flowtype: {flowtype}")
print() 
print(f"Time elapsed: {np.round(end-start,4)} seconds")
print() 
print(f"Error in h: {error_rate_h}")
print(f"Error in u: {error_rate_u}")


# plt.plot(x,q[0], label=0)
# for i in range (int(sim_time//dt)):
#     q0 = q_numerical[i+1]
#     h, u = q0 
#     plt.plot(x,h,label=i)
#     plt.legend()
# plt.show()
# %%
 

Start convergence test
N0: 0. Number of grid points: 16
some h: [nan nan nan nan nan]
N0: 1. Number of grid points: 32
some h: [nan nan nan nan nan]
N0: 2. Number of grid points: 64
some h: [nan nan nan nan nan]
N0: 3. Number of grid points: 128


/Users/rud/Documents/101/101-Math/101-Code/2024/Linearised shallow water wave equation/list_swwe_function.py:279: RuntimeWarning: invalid value encountered in true_divide
  u = uh/h
/Users/rud/Documents/101/101-Math/101-Code/2024/Linearised shallow water wave equation/list_swwe_function.py:315: RuntimeWarning: invalid value encountered in sqrt
  alphas = np.maximum(np.abs(u - np.sqrt(g*h)),np.abs(u + np.sqrt(g*h)))
/Users/rud/Documents/101/101-Math/101-Code/2024/Linearised shallow water wave equation/list_swwe_function.py:331: RuntimeWarning: invalid value encountered in true_divide
  RHS_uh = - I_N*(Q)@(uh**2/h + g*h**2/2) + (1/2)*A@uh


some h: [nan nan nan nan nan]
N0: 4. Number of grid points: 256
some h: [nan nan nan nan nan]
N0: 5. Number of grid points: 512
some h: [nan nan nan nan nan]
N0: 6. Number of grid points: 1024
some h: [nan nan nan nan nan]
N0: 7. Number of grid points: 2048
some h: [nan nan nan nan nan]
N0: 8. Number of grid points: 4096


KeyboardInterrupt: 

In [ ]:
error_rate_h

array([nan, nan, nan, nan, nan, nan, nan, nan])